## Setup

In [1]:
# imports and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE

In [2]:
# read in data as df
lending_df = pd.read_csv('Resources/lending_data.csv')
lending_df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


## Data Check

In [3]:
# check data types
lending_df.dtypes

loan_size           float64
interest_rate       float64
borrower_income       int64
debt_to_income      float64
num_of_accounts       int64
derogatory_marks      int64
total_debt            int64
loan_status           int64
dtype: object

In [4]:
# check nulls
lending_df.isna().sum()

loan_size           0
interest_rate       0
borrower_income     0
debt_to_income      0
num_of_accounts     0
derogatory_marks    0
total_debt          0
loan_status         0
dtype: int64

In [5]:
# check number of unique values
lending_df.nunique()

loan_size            182
interest_rate       4692
borrower_income      662
debt_to_income       662
num_of_accounts       17
derogatory_marks       4
total_debt           662
loan_status            2
dtype: int64

In [6]:
# check value counts of categorical col --> very uneven distribution of 0 and 1 labels
lending_df['loan_status'].value_counts()

loan_status
0    75036
1     2500
Name: count, dtype: int64

## Split Data: Training & Testing Sets
---

### Get y variable and x variables

In [7]:
# separate y and x cols
y_var = lending_df.iloc[:,-1]
x_vars = lending_df.iloc[:, :-1]

### Split into 75% training and 25% testing sets

#### no stratifying or balancing

In [8]:
# split into testing and training sets
x_train, x_test, y_train, y_test = train_test_split(x_vars, y_var)

#### preserve original proportion of 0s and 1s when splitting and balance training set

In [9]:
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(
    x_vars,
    y_var,
    stratify=y_var  #preserver the class ratio
)

In [10]:
# apply rebalancing on training set
sm = SMOTE(random_state=1)
x_train_bal, y_train_bal = sm.fit_resample(x_train_2, y_train_2)

## Create Logistic Regression Models
---

### Fit model with training data (regular)

In [11]:
# declare logistic regression model
logistic_regression_model = LogisticRegression(random_state=1)

# fit and save model
lr_model = logistic_regression_model.fit(x_train, y_train)

In [12]:
# get predicted valeus for testing and training
testing_predictions = logistic_regression_model.predict(x_test)

### Fit model with  training data (stratified and balanced)

In [13]:
# fit and save model
bal_lr_model = logistic_regression_model.fit(x_train_bal, y_train_bal)

In [14]:
# get predicted valeus for testing and training
testing_predictions_2 = logistic_regression_model.predict(x_test_2)

## Evaluate models
---

In [15]:
# define function for metrics: accuracy score, confusion matrix, and classification report
def get_scoring(y_set, predictions):
    # confusion matrix
    cm = confusion_matrix(y_set, predictions)
    cm_df = pd.DataFrame(
        cm, index=['Actual 0', 'Actual 1'], columns=['Predicted 0', 'Predicted 1']
    )

    # display all scores
    print(f'Accuracy Score: {accuracy_score(y_set, predictions)}\n')
    print('Confusion matrix')
    display(cm_df)
    print('\nClassification Report')
    print(classification_report(y_set, predictions))

### Not balanced model

In [16]:
get_scoring(y_test, testing_predictions)

Accuracy Score: 0.9922616591002889

Confusion matrix


,Predicted 0,Predicted 1
Actual 0,18678,100
Actual 1,50,556



Classification Report
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18778
           1       0.85      0.92      0.88       606

    accuracy                           0.99     19384
   macro avg       0.92      0.96      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Balanced model

In [17]:
get_scoring(y_test_2, testing_predictions_2)

Accuracy Score: 0.9954085843995047

Confusion matrix


,Predicted 0,Predicted 1
Actual 0,18671,88
Actual 1,1,624



Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     18759
           1       0.88      1.00      0.93       625

    accuracy                           1.00     19384
   macro avg       0.94      1.00      0.97     19384
weighted avg       1.00      1.00      1.00     19384



### How well does logistic regression model predict both the healthy loan (0) and high-risk loan (1)?

#### Not balanced model:
---

**Overall accuracy:** 99.26%

**Healthy loan (0):**
- ~100% of healthy loan predictions are correct
- ~99% of all healthy loans are identified
    
**High-risk loan (1):**
- ~85% of high risk loan predictions are correct
- ~92% of all high risk loans are identified

#### Balanced model:
---

**Overall accuracy:** 99.54%

**Healthy loan (0):**
- ~100% of healthy loan predictions are correct
- ~100% of all healthy loans are identified
    
**High-risk loan (1):**
- ~88% of high risk loan predictions are correct
- ~100% of all high risk loans are identified